# Agno Agents with Together AI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Tools_and_Integrations/Agent_Frameworks/Agno/Agents_Agno.ipynb)

<img src="../../../images/Agno.png" width="700">

## Introduction

Agno is a framework for building AI agents that can use tools, access knowledge bases, and work together in teams. This notebook demonstrates how to use Agno with Together AI's language models to create powerful AI applications with different capabilities.

In this notebook, we'll explore:
1. Setting up a simple agent with web search capability
2. Creating an agent with a knowledge base for specialized domains
3. Building multi-agent systems where specialized agents work together

## Install required packages for this notebook
- agno: The agent framework we'll be using
- duckduckgo-search: For web search capabilities
- pypdf: For processing PDF documents  
- lancedb and tantivy: For vector storage
- yfinance: For financial data retrieval

In [3]:
!pip install -qU agno duckduckgo-search pypdf lancedb tantivy yfinance openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dspy 2.6.10 requires datasets<3.0.0,>=2.14.6, but you have datasets 3.5.0 which is incompatible.


## Simple Agent

Below we create a basic agent that can search the web to answer questions. This agent uses:
- The Meta-Llama-3.1-8B-Instruct-Turbo
- DuckDuckGo search as a tool to retrieve information from the web

When we run this agent with a question about the NBA, it will search for recent information and provide an answer.

In [10]:
from agno.agent import Agent
from agno.models.together import Together
from agno.tools.duckduckgo import DuckDuckGoTools

agent = Agent(
    model=Together(id="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", max_tokens=4096),
    tools=[DuckDuckGoTools()],
    markdown=True
)
agent.print_response("How are the golden state warriors doing this 2024-2025 season?", stream=True)

Output()

# Agent with Knowledge Base

This example demonstrates how to create an agent with specialized knowledge. We'll build a Thai cuisine expert by:
1. Loading a PDF containing Thai recipes into a knowledge base
2. Setting up a vector database to store and retrieve this information efficiently
3. Configuring the agent to prioritize its knowledge base while still being able to search the web

This type of agent is ideal for specialized domains where you want to combine proprietary knowledge with the ability to find supplementary information.

In [13]:
# Import additional components for knowledge base functionality
from agno.agent import Agent
from agno.models.together import Together
from agno.embedder.together import TogetherEmbedder
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.lancedb import LanceDb, SearchType

# Create an agent with a specialized knowledge base for Thai recipes
agent = Agent(
    # Use Together AI's Llama 3.1 model
    model=Together(id="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", max_tokens=4096),
    # Define the agent's role
    description="You are a Thai cuisine expert!",
    # Give the agent specific instructions on how to use its knowledge
    instructions=[
        "Search your knowledge base for Thai recipes.",
        "If the question is better suited for the web, search the web to fill in gaps.",
        "Prefer the information in your knowledge base over the web results."
    ],
    # Set up the knowledge base from a PDF URL
    knowledge=PDFUrlKnowledgeBase(
        urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
        # Configure the vector database for semantic search
        vector_db=LanceDb(
            uri="tmp/lancedb",
            table_name="recipes",
            search_type=SearchType.hybrid,  # Use hybrid search (keyword + semantic)
            # Use Together AI's embedding model for vectorizing text
            embedder=TogetherEmbedder(id="togethercomputer/m2-bert-80M-32k-retrieval"),
        ),
    ),
    # Add web search capability as a backup
    tools=[DuckDuckGoTools()],
    # Display tool usage in the output
    show_tool_calls=True,
    markdown=True
)

# Load the knowledge base (only needs to be done once)
if agent.knowledge is not None:
    agent.knowledge.load()

agent.print_response("How do I make chicken and galangal in coconut milk soup", stream=True)
agent.print_response("What is the history of Thai curry?", stream=True)

INFO Loading knowledge base

INFO Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO No documents to insert

INFO Added 0 documents to knowledge base

Output()

Output()

# Multi-Agent System

This example demonstrates how to create a team of specialized agents that work together. We'll build:
1. A web search agent for finding general information
2. A finance agent for retrieving financial data
3. A coordinator agent that delegates tasks to the specialized agents

This approach allows us to create more powerful systems by combining specialized capabilities.

In [14]:
from agno.agent import Agent
from agno.models.together import Together
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.yfinance import YFinanceTools

# Create a specialized agent for web searches
web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    # Using the larger Llama 3.3 (70B) model for better performance
    model=Together(id="meta-llama/Llama-3.3-70B-Instruct-Turbo", max_tokens=4096),
    tools=[DuckDuckGoTools()],
    instructions="Always include sources",
    show_tool_calls=True,
    markdown=True,
)

# Create a specialized agent for financial data
finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=Together(id="meta-llama/Llama-3.3-70B-Instruct-Turbo", max_tokens=4096),
    # Financial tools for stock data, analyst recommendations, and company info
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions="Use tables to display data",
    show_tool_calls=True,
    markdown=True,
)

# Create a coordinator agent that manages the team
agent_team = Agent(
    # Provide the specialized agents as a team
    team=[web_agent, finance_agent],
    # The coordinator also uses a powerful model
    model=Together(id="meta-llama/Llama-3.3-70B-Instruct-Turbo", max_tokens=4096),
    # Instructions for the final output
    instructions=["Always include sources", "Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)

agent_team.print_response("What's the market outlook and financial performance of AI semiconductor companies?", stream=True)

Output()

In this notebook, we've demonstrated three approaches to building AI agents with Agno and Together AI:

1. **Simple Agent**: Quick to set up and useful for general tasks that require web search
2. **Knowledge Base Agent**: Combines specialized knowledge with web search for domain-specific applications
3. **Multi-Agent System**: Coordinates specialized agents for complex tasks requiring different capabilities

These examples show the flexibility of the Agno framework for building various types of AI applications. You can extend these examples by:
- Adding more tools to enhance capabilities
- Building larger knowledge bases for deeper expertise
- Creating more specialized agents for complex multi-agent systems